# Financial Instruments - Homework 7

## Part 1: American Options

**Binomial Tree Valuation**

In [ ]:
import numpy as np

S0 = 100.0
K = 100.0
u = 1.1
d = 1.0 / u
T = 3
r_cc = 0.02
p_rn = (np.exp(r_cc) - d) / (u - d)

def build_tree(S, u, d, N):
    tree = []
    for i in range(N + 1):
        level = [S * (u**j) * (d**(i-j)) for j in range(i + 1)]
        tree.append(level)
    return tree

price_tree = build_tree(S0, u, d, T)

def value_option(tree, K, r, p, type='call'):
    N = len(tree) - 1
    vals = []
    if type == 'call':
        vals = [max(s - K, 0) for s in tree[-1]]
    else:
        vals = [max(K - s, 0) for s in tree[-1]]
        
    for i in range(N - 1, -1, -1):
        new_vals = []
        for j in range(i + 1):
            cont = np.exp(-r) * (p * vals[j+1] + (1-p) * vals[j])
            s = tree[i][j]
            ex = max(s - K, 0) if type == 'call' else max(K - s, 0)
            new_vals.append(max(cont, ex))
        vals = new_vals
    return vals[0]

print(f"Call (r=2%): {value_option(price_tree, K, r_cc, p_rn, 'call'):.4f}")
print(f"Put (r=2%): {value_option(price_tree, K, r_cc, p_rn, 'put'):.4f}")

## Part 2: Citigroup KMV

**KMV Solver**

In [ ]:
from scipy.optimize import fsolve
from scipy.stats import norm

Deposits = 780.343
ShortTerm = 352.274
LongTerm = 396.097
Other = 395.693
D_im = Deposits + ShortTerm
L_opt = LongTerm + Other
K_opt = L_opt

# 10/10/2008
P1 = 13.9
Shares = 19.02 / 3.49
E1 = P1 * Shares
Sig_E1 = 0.75753
r = 0.02
T = 1.0

def kmv_sys(x, E_obs, Sig_E_obs, D_im, K, T, r):
    V, Sig_A = x
    V_adj = V - D_im
    d1 = (np.log(V_adj/K) + (r + 0.5*Sig_A**2)*T) / (Sig_A*np.sqrt(T))
    d2 = d1 - Sig_A*np.sqrt(T)
    E_mod = V_adj * norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    Sig_E_mod = norm.cdf(d1) * (V / E_obs) * Sig_A
    return [E_mod - E_obs, Sig_E_mod - Sig_E_obs]

root = fsolve(kmv_sys, [E1 + D_im + K_opt, Sig_E1 * E1/(E1+D_im+K_opt)], args=(E1, Sig_E1, D_im, K_opt, T, r))
V1, Sig_A1 = root
print(f"V1: {V1:.2f}, Sig_A1: {Sig_A1:.2%}")

def calc_pd(V, Sig_A, D_im, K, T, r):
    V_adj = V - D_im
    d2 = (np.log(V_adj/K) + (r - 0.5*Sig_A**2)*T) / (Sig_A*np.sqrt(T))
    return norm.cdf(-d2)

print(f"PD1: {calc_pd(V1, Sig_A1, D_im, K_opt, T, r):.4%}")